In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
# define raw data directory
data_path = './raw_data/'

# the car company list u want to download
car_com_lst = ['TSLA',
                'GM',
                'F']

In [3]:
# car_com_lst = ['BZ=F'] # to get oil future price

In [4]:
for carCom in car_com_lst:
    df = yf.download(carCom, 
                     start='2021-01-01', 
                     end='2023-03-25', 
                     # period = '5d',
                     progress=False)
    
    df.to_csv(data_path + carCom + "_20210101_to_20230325.csv")

In [7]:
TSLA_df = pd.read_csv(data_path + 'TSLA_20210101_to_20230325.csv', parse_dates=['Date'])
GM_df = pd.read_csv(data_path + 'GM_20210101_to_20230325.csv', parse_dates=['Date'])
F_df = pd.read_csv(data_path + 'F_20210101_to_20230325.csv', parse_dates=['Date'])
DJAHD_df = pd.read_csv(data_path + 'Dow Jones Automobiles & Parts Historical Data.csv', parse_dates=['Date'])
Brent_df = pd.read_csv(data_path + 'Europe_Brent_Spot_Price_FOB_modified.csv', parse_dates=['Day'])

In [8]:
DJAHD_df['Price'] = DJAHD_df['Price'].str.replace(',', '').astype(float)
DJAHD_df['Open'] = DJAHD_df['Open'].str.replace(',', '').astype(float)
DJAHD_df['High'] = DJAHD_df['High'].str.replace(',', '').astype(float)
DJAHD_df['Low'] = DJAHD_df['Low'].str.replace(',', '').astype(float)
DJAHD_df['Vol.'] = DJAHD_df['Vol.'].str.replace(',', '')
DJAHD_df['Vol.'] = DJAHD_df['Vol.'].str.replace('M', '').astype(float)
DJAHD_df['Vol.'] = DJAHD_df['Vol.']*1000000
DJAHD_df['Vol.'] = DJAHD_df['Vol.'].astype(int)
DJAHD_df['Change %'] = DJAHD_df['Change %'].str.replace('%', '').astype(float)

In [9]:
final_df = TSLA_df.set_index('Date')
final_df = final_df.join(GM_df.set_index('Date'), how='left', lsuffix='_TSLA', rsuffix='_GM')
final_df = final_df.join(F_df.set_index('Date'), how='left')
final_df = final_df.rename(columns={"Close": "Close_F", 
                                    "Open": "Open_F", 
                                    "High": "High_F", 
                                    "Low": "Low_F", 
                                    "Adj Close": "Adj Close_F",
                                    "Volume": "Volume_F"})

In [10]:
final_df = final_df.join(DJAHD_df.set_index('Date'), how='left')
final_df = final_df.rename(columns={"Price":"Price_DJAHD",
                                    "Open":"Open_DJAHD",
                                    "High":"High_DJAHD",
                                    "Low":"Low_DJAHD",
                                    "Vol.":"Vol._DJAHD",
                                    "Change %":"Change %_DJAHD"})

In [11]:
final_df = final_df.join(Brent_df.set_index('Day'), how='left')

In [12]:
final_df.dtypes

Open_TSLA                                          float64
High_TSLA                                          float64
Low_TSLA                                           float64
Close_TSLA                                         float64
Adj Close_TSLA                                     float64
Volume_TSLA                                          int64
Open_GM                                            float64
High_GM                                            float64
Low_GM                                             float64
Close_GM                                           float64
Adj Close_GM                                       float64
Volume_GM                                            int64
Open_F                                             float64
High_F                                             float64
Low_F                                              float64
Close_F                                            float64
Adj Close_F                                        float

In [13]:
final_df[final_df.isna().any(axis=1)]

,Open_TSLA,High_TSLA,Low_TSLA,Close_TSLA,Adj Close_TSLA,Volume_TSLA,Open_GM,High_GM,Low_GM,Close_GM,...,Close_F,Adj Close_F,Volume_F,Price_DJAHD,Open_DJAHD,High_DJAHD,Low_DJAHD,Vol._DJAHD,Change %_DJAHD,Europe Brent Spot Price FOB Dollars per Barrel
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-05,235.903336,236.053329,228.233337,230.350006,230.350006,125528400,59.000000,61.189999,58.750000,61.040001,...,12.700000,11.511685,82332700,1064.91,1073.68,1080.60,1055.99,154040000,4.08,NaN
2021-05-03,234.600006,235.333328,226.833328,228.300003,228.300003,81129300,57.599998,58.340000,56.709999,57.150002,...,11.630000,10.541803,61987300,1051.21,1071.84,1074.88,1046.16,106940000,-2.34,NaN
2021-08-30,238.240005,243.666672,237.576660,243.636673,243.636673,55812600,49.950001,49.950001,48.950001,49.169998,...,13.050000,11.828938,42720100,1084.18,1069.38,1085.13,1069.38,86500000,1.48,NaN
2021-12-27,357.890015,372.333344,356.906677,364.646667,364.646667,71145900,56.889999,57.750000,56.529999,57.430000,...,20.799999,18.948719,59651800,1529.11,1500.67,1552.03,1498.64,170590000,2.38,NaN
2021-12-28,369.829987,373.000000,359.473328,362.823334,362.823334,60324000,57.400002,58.119999,57.009998,57.110001,...,20.760000,18.912281,53020500,1520.29,1543.79,1551.50,1512.98,136920000,-0.58,NaN
2022-04-18,329.676666,338.306671,324.470001,334.763336,334.763336,51715200,40.130001,40.930000,39.950001,40.630001,...,15.670000,14.348704,40205800,1317.67,1300.64,1330.08,1283.63,117950000,1.52,NaN
2022-05-02,286.923340,302.119995,282.676666,300.980011,300.980011,75781500,37.669998,38.490002,37.250000,38.450001,...,14.270000,13.153690,64734000,1200.96,1152.07,1203.48,1137.96,157050000,3.22,NaN
2022-06-02,244.156662,264.209991,242.066666,258.333344,258.333344,93473100,38.400002,39.250000,38.360001,38.869999,...,13.890000,12.803418,42979700,1071.73,1023.34,1089.91,1016.59,137200000,4.04,NaN
2022-08-29,282.829987,287.739990,280.700012,284.820007,284.820007,41864700,38.770000,39.980000,38.700001,39.509998,...,15.570000,14.495132,50416500,1156.65,1146.81,1167.47,1142.79,134470000,-0.75,NaN


In [14]:
final_df = final_df.fillna(method='ffill')
final_df[final_df.isna().any(axis=1)]

,Open_TSLA,High_TSLA,Low_TSLA,Close_TSLA,Adj Close_TSLA,Volume_TSLA,Open_GM,High_GM,Low_GM,Close_GM,...,Close_F,Adj Close_F,Volume_F,Price_DJAHD,Open_DJAHD,High_DJAHD,Low_DJAHD,Vol._DJAHD,Change %_DJAHD,Europe Brent Spot Price FOB Dollars per Barrel
Date,,,,,,,,,,,,,,,,,,,,,


In [15]:
# save the file
final_df.to_pickle("stock_price_all.pkl")

In [16]:
final_df

,Open_TSLA,High_TSLA,Low_TSLA,Close_TSLA,Adj Close_TSLA,Volume_TSLA,Open_GM,High_GM,Low_GM,Close_GM,...,Close_F,Adj Close_F,Volume_F,Price_DJAHD,Open_DJAHD,High_DJAHD,Low_DJAHD,Vol._DJAHD,Change %_DJAHD,Europe Brent Spot Price FOB Dollars per Barrel
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,239.820007,248.163330,239.063339,243.256668,243.256668,145914600,41.950001,42.119999,40.139999,40.509998,...,8.52,7.722802,85043100,1030.48,1025.08,1050.03,1016.65,155890000,2.09,50.37
2021-01-05,241.220001,246.946671,239.733337,245.036667,245.036667,96735600,40.060001,41.840000,40.040001,41.660000,...,8.65,7.840637,70127800,1041.80,1022.67,1046.87,1020.23,124830000,1.10,53.16
2021-01-06,252.830002,258.000000,249.699997,251.993332,251.993332,134100000,41.919998,43.849998,41.860001,42.980000,...,8.84,8.012859,72590200,1072.15,1067.12,1093.94,1061.16,149400000,2.91,53.80
2021-01-07,259.209991,272.329987,258.399994,272.013336,272.013336,154496700,43.980000,43.980000,42.959999,43.320000,...,9.06,8.212275,77117100,1142.45,1098.67,1143.67,1098.33,149330000,6.56,53.70
2021-01-08,285.333344,294.829987,279.463318,293.339996,293.339996,225166500,43.310001,43.490002,42.320000,43.060001,...,9.00,8.157888,59162200,1209.78,1185.82,1216.75,1165.60,160450000,5.89,55.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-20,178.080002,186.440002,176.350006,183.250000,183.250000,129684400,33.450001,34.250000,33.270000,33.619999,...,11.18,11.180000,74975100,800.19,782.81,813.01,777.18,279270000,1.38,72.42
2023-03-21,188.279999,198.000000,188.039993,197.580002,197.580002,153391400,34.630001,35.320000,34.459999,35.099998,...,11.72,11.720000,74794900,852.26,820.48,853.69,820.48,299940000,6.51,74.78
2023-03-22,199.300003,200.660004,190.949997,191.149994,191.149994,150376400,35.130001,35.320000,34.029999,34.049999,...,11.48,11.480000,69729100,826.47,857.81,862.76,825.95,282970000,-3.03,76.08
